This notebook was motivated by

[2] Kaiming He et al. ‘Deep Residual Learning for Image Recognition’. In: CoRR abs/1512.03385 (2015). arXiv: 1512.03385.
url: http: //arxiv.org/abs/1512.03385.

Implementation: Oleh Bakumenko, University of Duisburg-Essen

# Imports

In [1]:
import sys
sys.path.append("../")
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import torch, torch.nn as nn
import torchvision, torchvision.transforms as tt
from torch.multiprocessing import Manager
torch.multiprocessing.set_sharing_strategy("file_system")
from torchsummary import summary
from pathlib import Path

from utility import utils as uu
from utility.eval import evaluate_classifier_model

from utility.trainLoopClassifier import *
from utility.plotImageModel import *

# Data augmentations

Data augmentation is a technique used to artificially increase the size of a dataset by transforming existing data points to create new, similar instances. This can help prevent overfitting in machine learning models, as well as improve their ability to generalize to unseen data. Common types of data augmentation include flipping, rotation, scaling, and adding noise to images.
We can generate the augmentation list with torchvision.transforms module


In [2]:
data_augments = torchvision.transforms.Compose([ 
    torchvision.transforms.RandomHorizontalFlip(p = .5),
    torchvision.transforms.RandomVerticalFlip(p = .5),
    torchvision.transforms.ColorJitter(brightness=(0.5,1.5), contrast=(1), hue=(-0.1,0.1)),
    #torchvision.transforms.RandomCrop((224, 224)),
    ])


Load the dataset from utils

In [3]:
cur_path = Path("plots_and_graphs.ipynb")
parent_dir = cur_path.parent.absolute()
masterThesis_folder = str(parent_dir.parent.absolute())+'/'
data_dir = masterThesis_folder+"data/Clean_LiTS/"

cache_me = False
if cache_me is True:
    cache_mgr = Manager()
    cache_mgr.data = cache_mgr.dict()
    cache_mgr.cached = cache_mgr.dict()
    for k in ["train", "val", "test"]:
        cache_mgr.data[k] = cache_mgr.dict()
        cache_mgr.cached[k] = False
# function from utils, credit: Institute for Artificial Intelligence in Medicine. url: https://mml.ikim.nrw/
# dataset outputs a tensor image (dimensions [1,256,256]) and a tensor target (0, 1 or 2)

ds = uu.LiTS_Classification_Dataset(
    data_dir=data_dir,
    transforms=data_augments,
    verbose=True,
    cache_data=cache_me,
    cache_mgr=(cache_mgr if cache_me is True else None),
    debug=True,
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3039 [00:00<?, ?it/s]

  0%|          | 0/3038 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3039 [00:00<?, ?it/s]

  0%|          | 0/3038 [00:00<?, ?it/s]

Dataset initialization complete.


# Hyperparameters

In [4]:
# Default settings
batch_size = 32
learning_rate = 1e-4
weight_decay = 1e-5
epochs = 15
run_name = "ResNet152"
device = ("cuda" if torch.cuda.is_available() else "cpu")
time_me  = True

The `torch.utils.data.DataLoader` is a utility class in PyTorch that makes the loading and batching of data for training purposes faster. It simplifies the process by allowing us to specify the dataset, batch size (often 32), and whether the data should be shuffled before each epoch. Additionally, there are other parameters available to further customize the data loading process.

In [5]:
# Dataloader
dl = torch.utils.data.DataLoader(
    dataset = ds, 
    batch_size = batch_size, 
    num_workers = 4, 
    shuffle = True, 
    drop_last = False, 
    pin_memory = True,
    persistent_workers = (not cache_me),
    prefetch_factor = 1
    )

ResNet (Residual Network) is a deep neural network architecture introduced in 2015 in [2]. It was designed to address the issue of vanishing gradients in very deep networks. ResNet is named as such because it utilizes residual connections (skip connections), which enable the flow of gradients from earlier layers to later layers, even in very deep networks.

The residual connections in ResNet involve adding the input of a layer to the output of a layer that is several layers deeper. This allows the network to more easily learn identity functions. This design helps prevent the issue of vanishing gradients and enables ResNet to train much deeper networks than was previously possible. This architecture has shown significant improvements in benchmarks compared to the earlier AlexNet model.

The original ResNet was used in the ImageNet Challenge to classify 1000 classes. However, in our exercise, we only use 3 classes:
0: Image does not include the liver.
1: Liver is visible.
2: Liver is visible and a lesion is visible.

ResNet50 introduced a new structure called the bottleneck block, which consists of a sequence of [convolution -> batch normalization -> activation] repeated three times. In addition to the 3x3 convolution, the number of channels is also modified within each block using a 1x1 convolution.

To provide more flexibility for customization, the variables "number of channels in," "between," and "out" are specified for the block. In most cases, the number of input channels will be the same as the number of output channels.

A generalized solution is used to address the downsampling issue. Instead of using two different blocks as in ResNet34, a boolean variable and a stride are defined for this purpose. If downsampling is required at the beginning of resblocks3-5, we set "downsample" to True and "stride" to 2. It's important to note that there is no need to change the stride in the second part of resblocks2.
If downsampling is not needed, the operation will be set to nn.Identity().

A few words about the torch.nn.init part:
PyTorch initializes the parameters for Conv and batch norm randomly from uniform distribution. Initialization of the weights and biases with a normal distribution helps the model backpropagate gradients in early epochs.

Tests were conducted on smaller models with 18, 34, and 50 layers, indicating that for adaptive optimizers, weight and bias initialization has minimal effect on model performance or convergence.

In contrast, the uniform initialized ResNet 152 model exhibited poor convergence after 15 epochs, with very high error and low accuracy rates. Although initialization improved the performance, it still required tuning of hyperparameters and a better optimizer.

In [6]:
# ResBlockBottleneck Class
#       - constructs a block [conv -> batch_norm -> activation]*3, which we will stack in the network
# Input:    int: num_chans_in, int:num_chans_between, int:num_chans_out
#           boolean: downsample = False, set True if first block
#           int: stride = 1, set 2 if want to downsample
# Output:   nn.Sequential() block
class ResBlockBottleneck(nn.Module):
    def __init__(self, num_chans_in,n_chans_between,num_chans_out, downsample = False, stride = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(num_chans_in, n_chans_between, kernel_size=1, padding=0, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(num_features=n_chans_between)
        self.relu = torch.nn.ReLU()
        self.conv2 = nn.Conv2d(n_chans_between, n_chans_between, kernel_size=3, stride= stride, padding=1, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(num_features=n_chans_between)
        self.relu = torch.nn.ReLU()
        self.conv3 = nn.Conv2d(n_chans_between, num_chans_out, kernel_size=1, padding=0, bias=False)
        self.batch_norm3 = nn.BatchNorm2d(num_features=num_chans_out)
        self.relu = torch.nn.ReLU()

        torch.nn.init.kaiming_normal_(self.conv1.weight,
                                      nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.conv2.weight,
                                      nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.conv3.weight,
                                      nonlinearity='relu')

        torch.nn.init.constant_(self.batch_norm1.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm1.bias)

        torch.nn.init.constant_(self.batch_norm2.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm2.bias)

        torch.nn.init.constant_(self.batch_norm3.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm3.bias)

        if downsample:
            self.downsample = nn.Sequential(
                nn.Conv2d(num_chans_in, num_chans_out, kernel_size=1,padding=0,stride=stride, bias=False),
                nn.BatchNorm2d(num_features=num_chans_out),
                nn.ReLU()
            )
        else:
            self.downsample = nn.Identity()

    def forward(self, x):
        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.batch_norm3(out)
        out = self.relu(out)
        return out + self.downsample(x)

# ResNet 152

The same as ResNet 50 but with more blocks

In [7]:
# ResNetMLMed152 Class
#       - constructs a ResNet152 as described in [2, Table 1].
# Input:    Tensor: [Batch,1,Height,Width]
# Output:   Tensor: [Batch,3]
class ResNetMLMed152(torch.nn.Module):
    def __init__(self):
        super(ResNetMLMed152, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size =7, stride =2, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(num_features=64)
        self.relu = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(kernel_size = 3, stride = 2,padding=1)
        self.resblocks2 = nn.Sequential(
            ResBlockBottleneck(num_chans_in = 64,n_chans_between =64 ,num_chans_out=256,downsample= True),
            *(2 * [ResBlockBottleneck(num_chans_in = 256,n_chans_between=64,num_chans_out= 256)]))
        self.resblocks3 = nn.Sequential(
            ResBlockBottleneck(num_chans_in = 256, n_chans_between=128, num_chans_out= 512, downsample=True, stride=2),
            *(7 * [ResBlockBottleneck(num_chans_in = 512,n_chans_between=128,num_chans_out= 512)]))
        self.resblocks4 = nn.Sequential(
            ResBlockBottleneck(num_chans_in = 512, n_chans_between=256, num_chans_out= 1024,downsample=True, stride=2),
            *(35 * [ResBlockBottleneck(num_chans_in = 1024,n_chans_between=256,num_chans_out= 1024)]))
        self.resblocks5 = nn.Sequential(
            ResBlockBottleneck(num_chans_in = 1024, n_chans_between=512, num_chans_out= 2048,downsample=True, stride=2),
            *(2 * [ResBlockBottleneck(num_chans_in = 2048,n_chans_between=512,num_chans_out= 2048)]))
        self.avgpool6 = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=2048, out_features=3, bias=True)

    def forward(self, x):
        out_1 = self.conv1(x)
        out_1 = self.batch_norm1(out_1)
        out_1 = self.relu(out_1)
        out_1 = self.pool2(out_1)

        out_2 = self.resblocks2(out_1)

        out_3 = self.resblocks3(out_2)

        out_4= self.resblocks4(out_3)

        out_5= self.resblocks5(out_4)

        out_6 = self.avgpool6(out_5)

        out_6= self.fc(torch.flatten(out_6, start_dim=1))

        return out_6

In [8]:
model = ResNetMLMed152()
summary(model, (1, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 126, 126]           3,200
       BatchNorm2d-2         [-1, 64, 126, 126]             128
              ReLU-3         [-1, 64, 126, 126]               0
         MaxPool2d-4           [-1, 64, 63, 63]               0
            Conv2d-5           [-1, 64, 63, 63]           4,096
       BatchNorm2d-6           [-1, 64, 63, 63]             128
              ReLU-7           [-1, 64, 63, 63]               0
            Conv2d-8           [-1, 64, 63, 63]          36,864
       BatchNorm2d-9           [-1, 64, 63, 63]             128
             ReLU-10           [-1, 64, 63, 63]               0
           Conv2d-11          [-1, 256, 63, 63]          16,384
      BatchNorm2d-12          [-1, 256, 63, 63]             512
             ReLU-13          [-1, 256, 63, 63]               0
           Conv2d-14          [-1, 256,

In [ ]:
for step, (data, targets) in enumerate(dl):
    data, targets = data.to(device), targets.to(device)
    if step ==1:
        break
model = model.to(device)
model(data).shape

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
criterion = nn.CrossEntropyLoss()

In [ ]:
training_loop_conf_matr(
    epochs = epochs,
    optimizer = optimizer,
    model = model,
    criterion = criterion,
    ds = ds,
    dl = dl,
    batch_size = batch_size,
    run_name = run_name,
    device = device,
    time_me=True,
    time=time)